# Recipient Country Counts

This notebook assesses volume of refugees by recipient country based on travel duration

In [1]:
import json
import pandas as pd

In [2]:
crossings_df = pd.read_csv('inputs/Ukraine_BorderCrossings.csv')

In [3]:
crossings_df.rename(columns={"Lat":"lat", "Long":"lon", "Name - English":"name"},inplace=True)

In [4]:
crossings_df = crossings_df[['name','Country','lat','lon']].dropna()
print(crossings_df.Country.unique())

# select only target countries, not Belarus or Russia
crossings_df = crossings_df[crossings_df['Country'].str.contains('Poland|Moldova|Romania|Slovakia|Hungary')]
print(crossings_df.Country.unique())

['Poland' 'Moldova' 'Romania ' 'Romania' 'Slovakia' 'Hungary ' 'Belarus'
 'Russian Federation']
['Poland' 'Moldova' 'Romania ' 'Romania' 'Slovakia' 'Hungary ']


## Read in Locations

In [5]:
df = pd.read_csv('inputs/ukraine_locations.csv')
df.head()

,#name,country,latitude,longitude,location_type,conflict_date,population
0,Horlivka,Ukraine,48.33576,38.05325,conflict_zone,0.0,278550.0
1,Mariupol,Ukraine,47.09514,37.54131,conflict_zone,0.0,481626.0
2,Kadiyivka,Ukraine,48.56818,38.64352,conflict_zone,1.0,84425.0
3,Schastia,Ukraine,48.74120,39.23540,conflict_zone,1.0,11743.0
4,Donetsk,Ukraine,48.02300,37.80224,conflict_zone,2.0,1024700.0


In [6]:
conflicts = df[df['location_type']=='conflict_zone']

In [7]:
conflicts.shape

(53, 7)

In [8]:
with open('outputs/ukraine_exit_routes.json','r') as f:
    conflict_exit_routes = json.loads(f.read())
    
with open('outputs/ukraine_exit_routes_transit.json','r') as f:
    conflict_exit_routes_transit = json.loads(f.read())    

In [9]:
def get_exit_route(row, mode):
    if mode == 'driving':
        routes = conflict_exit_routes
    elif mode == 'transit':
        routes = conflict_exit_routes_transit
    
    try:
        dest = routes[row['#name']]['crossing']['Country']
    except:
        dest = None
    row[f'{mode}_destination'] = dest
    return row

In [10]:
conflicts = conflicts.apply(lambda row: get_exit_route(row, 'driving'), axis=1)
conflicts = conflicts.apply(lambda row: get_exit_route(row, 'transit'), axis=1)

In [11]:
def transit_mixed(row):
    if row.transit_destination == None:
        dest = row.driving_destination
    else:
        dest = row.transit_destination
    row['mixed_destination'] = dest
    return row

In [12]:
conflicts = conflicts.apply(lambda row: transit_mixed(row), axis=1)

In [13]:
conflicts.head()

,#name,country,latitude,longitude,location_type,conflict_date,population,driving_destination,transit_destination,mixed_destination
0,Horlivka,Ukraine,48.33576,38.05325,conflict_zone,0.0,278550.0,Moldova,None,Moldova
1,Mariupol,Ukraine,47.09514,37.54131,conflict_zone,0.0,481626.0,Moldova,None,Moldova
2,Kadiyivka,Ukraine,48.56818,38.64352,conflict_zone,1.0,84425.0,Moldova,Poland,Poland
3,Schastia,Ukraine,48.74120,39.23540,conflict_zone,1.0,11743.0,Moldova,None,Moldova
4,Donetsk,Ukraine,48.02300,37.80224,conflict_zone,2.0,1024700.0,Moldova,None,Moldova


In [14]:
df = pd.read_csv('inputs/unhcr_refugee_counts_4.11.21.csv')

In [15]:
df.Population = df.Population.apply(lambda x: int(x.replace(',','')))

In [16]:
df

,Location name,Source,Data date,Population
0,Poland,Government,10 Apr 2022,2622117
1,Romania,Government,10 Apr 2022,692501
2,Hungary,Government,10 Apr 2022,424367
3,Republic of Moldova,Government,10 Apr 2022,411365
4,Russian Federation,Government,9 Apr 2022,404418
5,Slovakia,Government,10 Apr 2022,317781
6,Belarus,Government,10 Apr 2022,20739


In [17]:
df[~df['Location name'].isin(['Russian Federation','Belarus'])]

,Location name,Source,Data date,Population
0,Poland,Government,10 Apr 2022,2622117
1,Romania,Government,10 Apr 2022,692501
2,Hungary,Government,10 Apr 2022,424367
3,Republic of Moldova,Government,10 Apr 2022,411365
5,Slovakia,Government,10 Apr 2022,317781


In [18]:
ref_total = df[~df['Location name'].isin(['Russian Federation','Belarus'])].Population.sum()
conflict_total_driving = int(conflicts[conflicts['driving_destination'].notnull()].population.sum())
conflict_total_transit = int(conflicts[conflicts['transit_destination'].notnull()].population.sum())
conflict_total_mixed = int(conflicts[conflicts['mixed_destination'].notnull()].population.sum())

In [19]:
conflicts['pop_adjusted_driving'] = conflicts.population * (ref_total/conflict_total_driving)
conflicts['pop_adjusted_transit'] = conflicts.population * (ref_total/conflict_total_transit)
conflicts['pop_adjusted_mixed'] = conflicts.population * (ref_total/conflict_total_mixed)

In [21]:
pd.DataFrame(conflicts.groupby(['driving_destination'])['pop_adjusted_driving'].sum()).round()

,pop_adjusted_driving
driving_destination,
Moldova,4340796.0
Poland,98252.0
Romania,29083.0


In [22]:
pd.DataFrame(conflicts.groupby(['transit_destination'])['pop_adjusted_transit'].sum()).round()

,pop_adjusted_transit
transit_destination,
Moldova,3255544.0
Poland,1212587.0


In [23]:
pd.DataFrame(conflicts.groupby(['mixed_destination'])['pop_adjusted_mixed'].sum()).round()

,pop_adjusted_mixed
mixed_destination,
Moldova,3589553.0
Poland,878578.0


In [26]:
ukr_model_results = pd.read_csv('outputs/ukraine_model_results.csv')

In [27]:
ukr_model_results

,country,percent_IndividualPerCountry_of_recorded,predicted_shares
0,Hungary,0.094222,0.256627
1,Moldova,0.093682,0.170117
2,Poland,0.586410,0.388978
3,Romania,0.154561,0.092945
4,Slovakia,0.071125,0.052048


In [28]:
transit_res = pd.DataFrame(conflicts.groupby(['transit_destination'])['pop_adjusted_transit'].sum()).round().reset_index()

In [29]:
transit_res = transit_res.rename(columns={'transit_destination': 'country'})

In [30]:
results = ukr_model_results[['country','predicted_shares']]

In [31]:
results

,country,predicted_shares
0,Hungary,0.256627
1,Moldova,0.170117
2,Poland,0.388978
3,Romania,0.092945
4,Slovakia,0.052048


In [32]:
transit_res

,country,pop_adjusted_transit
0,Moldova,3255544.0
1,Poland,1212587.0


In [33]:
transit_res['transit_predicted_shares'] = transit_res.pop_adjusted_transit/transit_res.pop_adjusted_transit.sum()

In [34]:
transit_res

,country,pop_adjusted_transit,transit_predicted_shares
0,Moldova,3255544.0,0.728614
1,Poland,1212587.0,0.271386


In [35]:
results = pd.merge(results, transit_res, left_on='country', right_on='country', how='left')
results = results.fillna(0)

In [36]:
results = results[['country','predicted_shares','transit_predicted_shares']]

In [37]:
df_ = df[~df['Location name'].isin(['Russian Federation','Belarus'])]

In [38]:
df_ = df_.replace('Republic of Moldova','Moldova')\
    .rename(columns={'Location name': 'country', 'Population': 'refugees_actual'})[['country','refugees_actual']]

In [39]:
results = pd.merge(results,df_,how='left',left_on='country',right_on='country')

In [40]:
def naive_weighted(row, total_refugees):
    predicted = (row.predicted_shares*.5 + row.transit_predicted_shares*.5)*total_refugees
    row['refugees_predicted'] = int(predicted.round())
    return row

In [41]:
def attraction_only(row, total_refugees):
    predicted = (row.predicted_shares*total_refugees)
    row['refugees_predicted_attactions'] = int(predicted.round())
    return row

In [42]:
results

,country,predicted_shares,transit_predicted_shares,refugees_actual
0,Hungary,0.256627,0.000000,424367
1,Moldova,0.170117,0.728614,411365
2,Poland,0.388978,0.271386,2622117
3,Romania,0.092945,0.000000,692501
4,Slovakia,0.052048,0.000000,317781


In [43]:
total_refugees = results.refugees_actual.sum()
results = results.apply(lambda row: naive_weighted(row, total_refugees), axis=1)
results = results.apply(lambda row: attraction_only(row, total_refugees), axis=1)

In [44]:
results

,country,predicted_shares,transit_predicted_shares,refugees_actual,refugees_predicted,refugees_predicted_attactions
0,Hungary,0.256627,0.000000,424367,573321,1146642
1,Moldova,0.170117,0.728614,411365,2007825,760105
2,Poland,0.388978,0.271386,2622117,1475295,1738004
3,Romania,0.092945,0.000000,692501,207645,415290
4,Slovakia,0.052048,0.000000,317781,116278,232556
